# 

In [2]:
import pandas as pd
import numpy as np
from skimpy import clean_columns
import matplotlib.pyplot as plt

In [3]:
def read_csv(filename):
    df = pd.read_csv(f"dat/{filename}")
    df = clean_columns(df)
    return df

sb_ratings = read_csv('super-bowl-ratings.csv')
sb_dat = read_csv('superbowl_2.csv')
sb_off_stats = read_csv('superbowloffensivestats.csv')
sb_mvps = read_csv('mvpsb.csv')

to_short_name = {
    'Baltimore Colts': "IND",
    'Baltimore Ravens': "BAL",
    'Chicago Bears': "CHI",
    'Dallas Cowboys': "DAL",
    'Denver Broncos': "DEN",
    'Green Bay Packers': "GNB",
    'Indianapolis Colts': "IND",
    'Kansas City Chiefs':"KAN",
    'Los Angeles Raiders': "LAR",
    'Los Angeles Rams': "RAM", 
    'Miami Dolphins': "MIA",
    'New England Patriots': "NWE",
    'New Orleans Saints': "NOR",
    'New York Giants': "NYG",
    'New York Jets': "NYJ",
    'Oakland Raiders': "OAK",
    'Philadelphia Eagles': "PHI",
    'Pittsburgh Steelers': "PIT",
    'San Francisco 49ers': "SFO",
    'Seattle Seahawks': "SEA",
    'St. Louis Rams': "RAM",
    'Tampa Bay Buccaneers': "TAM",
    'Washington Redskins': "WAS",
    'Arizona Cardinals' : "ARI",
    'Atlanta Falcons': "ATL",
    'Buffalo Bills': "BUF",
    'Carolina Panthers': "CAR",
    'Cincinnati Bengals': "CIN",
    'Minnesota Vikings': "MIN",
    'San Diego Chargers': "SDG",
    'Tennessee Titans': "TEN"
}
to_short_names_mvps = {
    'Baltimore' : "BAL",
    'Chicago' : "CHI",
    'Dallas': "DAL",
    'Denver': "DEN",
    'Green Bay': "GNB",
    'Indianapolis': "IND",
    'Kansas City': "KAN",
    'Los Angeles Raiders': "LAR",
    'Miami': "MIA",
    'New England': "NWE",
    'New Orleans': "NOR",
    'New York Giants': "NYG",
    'New York Jets': "NYJ",
    'Oakland': "OAK",
    'Philadelphia': "PHI",
    'Pittsburgh': "PIT",
    'San Francisco': "SFO",
    'Seattle': "SEA",
    'St. Louis': "RAM",
    'Tampa Bay': "TAM",
    'Washington': "WAS"
}

sb_dat.replace(to_replace=to_short_name.keys(), value=to_short_name.values(), inplace=True)
sb_mvps.replace(to_replace=to_short_names_mvps.keys(), value=to_short_names_mvps.values(), inplace=True)
print(sb_off_stats.columns)

Index(['year', 'team', 'completions', 'pass_attempts', 'passing_yards',
       'passing_td', 'interception', 'times_sacked', 'sack_yards',
       'rush_attempts', 'rushing_td', 'receptions', 'receiving_yards',
       'receiving_td', 'fumbles_lost', 'winner', 'rushing_yards',
       'pass_completion', 'turnovers', 'yards_per_attempt'],
      dtype='object')


In [4]:
# group data
#sb_off_stats = sb_off_stats.groupby(["year", "team"])
import csv

sb_off_stats["pass_completion"] = sb_off_stats["completions"] / sb_off_stats["pass_attempts"]
sb_off_stats["turnovers"] = sb_off_stats["interception"] + sb_off_stats["fumbles_lost"]
sb_off_stats["yards_per_attempt"] = sb_off_stats["passing_yards"] / sb_off_stats["pass_attempts"]

sb_off_stats = sb_off_stats[["year", "team", "winner", "pass_completion", "turnovers", "yards_per_attempt", "rushing_yards", "times_sacked"]]

for col in sb_off_stats.columns:
    sb_off_stats[col] = sb_off_stats[col].fillna(0)
print(sb_off_stats)

sb_off_stats.to_csv('stats.csv', index=False)

     year team  winner  pass_completion  turnovers  yards_per_attempt  \
0      56  CIN       0         0.676471        0.0           7.911765   
1      56  LAR       1         0.634146        2.0           6.902439   
2      55  KAN       0         0.530612        2.0           5.510204   
3      55  TAM       1         0.724138        0.0           6.931034   
4      54  KAN       1         0.619048        2.0           6.809524   
..    ...  ...     ...              ...        ...                ...   
107     3  NYJ       1         0.586207        1.0           7.103448   
108     2  GNB       1         0.541667        0.0           8.416667   
109     2  OAK       0         0.441176        1.0           6.117647   
110     1  GNB       1         0.666667        1.0          10.416667   
111     1  KAN       0         0.531250        1.0           7.125000   

     rushing_yards  times_sacked  
0               79             7  
1               43             2  
2              107

In [5]:
# import matplotlib


# matplotlib.rcParams.update({'xtick.color': "white",
#                      'ytick.color': "white",
#                      'axes.labelcolor': "white"}) 
# bg = plt.imread("nfl.jpg")

# width = 10
# fig, axs = plt.subplots(nrows=1, ncols=4, figsize=(width, width * bg.shape[0] / bg.shape[1]))
# for ax in np.ravel(axs):
#     ax.patch.set_alpha(0.7)
x_train = sb_off_stats.head(82)
x_train = x_train.drop(columns=["winner", "year", "team"])
y_train = sb_off_stats["winner"].head(82)

x_val = sb_off_stats.tail(30)
x_val = x_val.drop(columns=["winner", "year", "team"])
y_val = sb_off_stats["winner"].tail(30)

import tensorflow as tf
from tensorflow import keras

# Define the model
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(5,)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', run_eagerly=True)

# Train the model
model.fit(x_train, y_train, epochs=50)

2023-11-04 15:49:40.255714: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-04 15:49:40.425404: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-04 15:49:40.426182: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-04 15:49:41.621975: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Epoch 1/50


2023-11-04 15:50:05.825336: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


3/3 [==============================] - 0s 25ms/step - loss: 5.5690
Epoch 2/50
3/3 [==============================] - 0s 24ms/step - loss: 1.1949
Epoch 3/50
3/3 [==============================] - 0s 25ms/step - loss: 2.0819
Epoch 4/50
3/3 [==============================] - 0s 26ms/step - loss: 2.2637
Epoch 5/50
3/3 [==============================] - 0s 24ms/step - loss: 1.3140
Epoch 6/50
3/3 [==============================] - 0s 25ms/step - loss: 0.9848
Epoch 7/50
3/3 [==============================] - 0s 25ms/step - loss: 1.2050
Epoch 8/50
3/3 [==============================] - 0s 25ms/step - loss: 0.6938
Epoch 9/50
3/3 [==============================] - 0s 25ms/step - loss: 0.9104
Epoch 10/50
3/3 [==============================] - 0s 26ms/step - loss: 0.8284
Epoch 11/50
3/3 [==============================] - 0s 28ms/step - loss: 0.5477
Epoch 12/50
3/3 [==============================] - 0s 27ms/step - loss: 0.7981
Epoch 13/50
3/3 [==============================] - 0s 25ms/step - loss: 

In [6]:
# Predict model
validation_predictions = model.predict(x_val)
for i in range(5):
    print(f"Model Predicted Win%: {validation_predictions[i]} i.e. {np.round(validation_predictions[i])}")
    print(f"Ground Truth: {y_val.iloc[i]}")

correct = 0

for i in range(30):
    if np.round(validation_predictions[i]) == y_val.iloc[i]:
        correct += 1

print(f"The model predicts the winner of the superbowl correctly: {int(correct/30 * 100)}% of the time")

1/1 [==============================] - 0s 15ms/step
Model Predicted Win%: [0.43024448] i.e. [0.]
Ground Truth: 0
Model Predicted Win%: [0.8829393] i.e. [1.]
Ground Truth: 1
Model Predicted Win%: [0.9080139] i.e. [1.]
Ground Truth: 1
Model Predicted Win%: [0.59236836] i.e. [1.]
Ground Truth: 0
Model Predicted Win%: [0.4322124] i.e. [0.]
Ground Truth: 0
The model predicts the winner of the superbowl correctly: 76% of the time


In [7]:
import tensorflow as tf
x_chiefs = tf.constant([[25/36, 2, 11.3, 112, 1]])
x_49ers = tf.constant([[29/46, 3, 6.7, 101, 3]])

chiefs_prediction = model.predict(x_chiefs)
forty_niners_prediction = model.predict(x_49ers)

print("Chiefs' prediction:", chiefs_prediction)
print("49ers' prediction:", forty_niners_prediction)


1/1 [==============================] - 0s 23ms/step
Chiefs' prediction: [[0.7276007]]
49ers' prediction: [[0.33228266]]


In [9]:
# What if they keep playing liek they have been?
x_average_chiefs = tf.constant([[212/310, 11/8, 7.4,841/8, 10/8]])
avg_chiefs_prediction = model.predict(x_average_chiefs)

print(avg_chiefs_prediction)

1/1 [==============================] - 0s 20ms/step
[[0.6514643]]
